In [16]:
import numpy as np

from scipy.integrate import odeint
from scipy import stats
import scipy.linalg as lin

import matplotlib.pyplot as plt
from matplotlib import pyplot
import pylab as plt

In [17]:
# Constant parameters

# Nersnt reversal potentials (mV)
E_Na = 72
E_K = -12
E_L = -60

# Leakage channel conductance (mS/cm^2)
gbar_L = 0.0328
    
# Membrane capacitance (uF/cm^2)
C_m = 0.77
    
# Sodium maximum channel conductance (mS/cm^2)
gbar_Na = 120

# Potassium maximum channel conductance (mS/cm^2)
gbar_K = 36

In [18]:
def hodgkinhuxley(state,t):
    v = state[0]
    y = state[1]
    m = state[2]
    h = state[3]

    # Dependent parameters
    # Sodium channel conductance
    gbar_Na * h * m**3

    # Potassium channel conductance
    g_K = gbar_K * (n**4)

In [20]:
def a_m(V): return 0.1*(V+40.0)/(1.0 - np.exp(-(V+40.0) / 10.0))
def b_m(V):  return 4.0*np.exp(-(V+65.0) / 18.0)
def a_h(V): return 0.07*np.exp(-(V+65.0) / 20.0)
def b_h(V):  return 1.0/(1.0 + np.exp(-(V+35.0) / 10.0))
def a_n(V): return 0.01*(V+55.0)/(1.0 - np.exp(-(V+55.0) / 10.0))
def b_n(V):  return 0.125*np.exp(-(V+65) / 80.0)

'''
# Channel Gating Kinetics for Sodium (Na) Channel m
# m is the activation variable for the Sodium (Na) Channel

def a_m(v): return 0.1*(25-v)/(np.exp((25-v)/10)-1)
def b_m(v): return 4.0*np.exp(-v/18)
    
# Channel Gating Kinetics for Sodium (Na) Channel h
# h is the inactivation variable for the Sodium (Na) Channel
def a_h(v): return 0.07*np.exp(-v/20)
def b_h(v): return 1.0/(np.exp((30-v)/10)+1)    

# Channel Gating Kinetics for Potassium (K) channel n
# n is the activation variable for the Potassium (K) Channel
# Potassium channel does not inactivate, so there is no inactivation variable
def a_n(v): return 0.01*(10-v)/(np.exp((10-v)/10)-1)
def b_n(v): return 0.125*np.exp(-v/80)
'''

'\n# Channel Gating Kinetics for Sodium (Na) Channel m\n# m is the activation variable for the Sodium (Na) Channel\n\ndef a_m(v): return 0.1*(25-v)/(np.exp((25-v)/10)-1)\ndef b_m(v): return 4.0*np.exp(-v/18)\n    \n# Channel Gating Kinetics for Sodium (Na) Channel h\n# h is the inactivation variable for the Sodium (Na) Channel\ndef a_h(v): return 0.07*np.exp(-v/20)\ndef b_h(v): return 1.0/(np.exp((30-v)/10)+1)    \n\n# Channel Gating Kinetics for Potassium (K) channel n\n# n is the activation variable for the Potassium (K) Channel\n# Potassium channel does not inactivate, so there is no inactivation variable\ndef a_n(v): return 0.01*(10-v)/(np.exp((10-v)/10)-1)\ndef b_n(v): return 0.125*np.exp(-v/80)\n'

In [21]:
# Membrane currents (in uA/cm^2)
# Sodium 
def I_Na(v,m,h):return g_Na * (v - E_Na)
# Potassium 
def I_K(v, n):  return g_K * (v - E_K)
# Leak
I_L = gbar_L

'''
I_Na = g_Na * (v-E_Na)
I_K = g_K * (v-E_K)
I_L = gbar_L 
I_inj = 10
'''

'\nI_Na = g_Na * (v-E_Na)\nI_K = g_K * (v-E_K)\nI_L = gbar_L \nI_inj = 10\n'

In [22]:
# Time to integrate over
t = np.arange(0,30,.00001)

In [23]:
# Integrate
def dALLdt(X, t):
    v, m, h, n = X
    
    # Calculate membrane potential & activation variables
    # Differential equations
    dvdt = (-I_Na(v,m,h) - I_K(v,n) - I_L + I_inj)/C_m
    dmdt = a_m(v)*(1.0-m) - b_m(v)*m
    dhdt = a_h(v)*(1.0-h) - b_h(v)*h
    dndt = a_n(v)*(1.0-n) - b_n(v)*n
    return dvdt, dndt, dmdt, dhdt

In [24]:
state0 = [7, 0.19, 0.18, 0.6]
X = odeint(dALLdt, state0, t)
v = X[:,0]
m = X[:,1]
h = X[:,2]
n = X[:,3]
ina = I_Na(v,m,h)
ik = I_K(v,n)
il = I_L(v)

NameError: name 'g_Na' is not defined

In [25]:
state0 = [7, 0.19, 0.18, 0.6]
state = odeint(hodgkinhuxley, state0, t)
plt.plot(t,state[:,0]-65)
pyplot.xlabel('Time (ms)')
pyplot.ylabel('Membrane Potential (mV)')
plt.show()

NameError: name 'n' is not defined

In [ ]:
t = np.arange (0,30,.00001)
state0 = [7, 0.19, 0.18, 0.6]
state = odeint(hodgkinhuxley, state0, t)
plt.plot(t,state[:,0]-65)
plt.plot(t, I_Na, 'c', label='$I_{Na}$')
plt.plot(t, I_K, 'y', label='$I_{K}$')
plt.plot(t, I_L, 'm', label='$I_{L}$')
plt.legend()
pyplot.xlabel('Time (ms)')
pyplot.ylabel('Current (uA)')
plt.show()

In [9]:
'''
Channel nernst reversal potentials
e_Na from 115 to 72: two spikes in 30 ms time span to 1; enough repolarization and ending potential at ~-65mV; peak at 
~10mV instead of 40mV
e_K from -12 to -72: almost like a damped sine waveform; first repolarizes then follows in pattern and amplitude decreases
e_L from 10.6 to -60: no change

Sodium maximum channel conductance
gbar_Na from 120 to 0.4: severe decrease in change in membrane potential (~4mV); still looks like an action potential with 
sufficient depolarization and repolarization
g_Na is function of gbar_Na, h, and m

Potassium maximum channel conductance
gbar_K from 36 to 0.37: regular change in membrane potential; sudden change at peak; doesn't repolarize enough - ends at 0mV
g_K is function of gbar_K and y

Leakage channel conductance
gbar_L from 0.3 to 0.0328: no change

Membrane capacitance
C_m from 1 to 0.77: no change

Channel currents
I_Na is function of g_Na, v, V_Na
I_K is function of g_K, v, V_K
I_L is function of g_L
I_app from 6.2 to 10: no change

state0 first term from 0 to 7: decrease in time span, slight increase in resting potential; as values increase more abnormal 
behavior
state0 second term: any small change results in different curves at peak any sizeable change results in abnormal damped 
behavior; slighly increased membrane potential at peak
state0 third term: decreases time span but makes peak slighly more of a sudden change
state0 fourth term: slight changes in peak shape; larger changes in membrane potential

Changed parameters:
E_Na from 115 to 72
E_L from 10.6 to -60
gbar_L from 0.3 to 0.0328
C_m from 1 to 0.77
I_app from 6.2 to 10
n from 0.28 to 0.19
m from 0.05 to 0.18
h from 0.6 to 0.28
'''

"\nChannel nernst reversal potentials\nE_Na from 115 to 72: two spikes in 30 ms time span to 1; enough repolarization and ending potential at ~-65mV; peak at \n~10mV instead of 40mV\nE_K from -12 to -72: almost like a damped sine waveform; first repolarizes then follows in pattern and amplitude decreases\nE_L from 10.6 to -60: no change\n\nSodium maximum channel conductance\ngbar_Na from 120 to 0.4: severe decrease in change in membrane potential (~4mV); still looks like an action potential with \nsufficient depolarization and repolarization\ng_Na is function of gbar_Na, h, and m\n\nPotassium maximum channel conductance\ngbar_K from 36 to 0.37: regular change in membrane potential; sudden change at peak; doesn't repolarize enough - ends at 0mV\ng_K is function of gbar_K and y\n\nLeakage channel conductance\ngbar_L from 0.3 to 0.0328: no change\n\nMembrane capacitance\nC_m from 1 to 0.77: no change\n\nChannel currents\nI_Na is function of g_Na, v, V_Na\nI_K is function of g_K, v, V_K\nI